# To run this script, you will need your user token (generate one if you don't have it).
https://github.com/settings/tokens

And save it as a file called _github_token.txt_ in a folder called _reusable_code_ on the Jupyter home

In [1]:
import github
import requests
import re

In [2]:
# Pick up Github token for authentication purposes 
mytoken=open('/home/jupyter/reusable_code/github_token.txt',"r").read()

In [3]:
#Create GitHub connection/ instance
g = github.Github(mytoken)

# Get your specific user (update with your username)
myuser=g.get_user('sw-2019')

# Subset to just the ITV repo
ITV_org=g.get_organization('ITV').get_repos()

# Subset further to just a list of BritBox DIG repos according to our naming convention
bbdig_repos=[i for i in ITV_org if i.full_name.replace('ITV/','')[:5]=='bbdig']

# Get the bb_insight team
bbinsightteam=g.get_organization('ITV').get_team([x.id for x in g.get_organization('ITV').get_teams() if x.name=='bb_insight'][0])


### Loop all repos in bbdig_repos that you have admin access to, and add the team if they aren't there already

In [ ]:
for r in bbdig_repos: # For each BBDIG repo
    try:
        amICollobarator=False
        for x in r.get_collaborators(): # Loop collaborators
            if x.login==myuser.login: # If I'm a collaborator
                    amICollobarator=True
                    if x.permissions.admin==True: #And I have admin permissions
                        Madechanges=False
                        if bbinsightteam.get_repo_permission(r)==None: #Check if the insight team has been added at all (None='No')
                            bbinsightteam.add_to_repos(r) #Check if the team is not added at all (None='No')
                            Madechanges=True
                            bbinsightteam.set_repo_permission(repo=r,permission='admin') # And make them admin
                            print('Added BB Insight team to {}'.format(r.name))
                        else:
                            if bbinsightteam.get_repo_permission(r).admin==False: #If team already exists but isn't admin
                                bbinsightteam.set_repo_permission(repo=r,permission='admin') # Make them admin
                                Madechanges=True
                                print('Made BB Insight team admins on {}'.format(r.name))
                        if Madechanges==False:
                            print('I can edit {} but no changes were required, team is already admin'.format(r.name))
                    else:
                        print('No admin rights on {}'.format(r.name))

        if amICollobarator==False:
            print('I''m not a collaborator on {}'.format(r.name))
    except:
        print('Cannot influence {}'.format(r.name))


In [39]:
#Subset

repo1=bbdig_repos[11]
repo1.name
#help(repo1)
#add_to_collaborators(self, collaborator, permission=NotSet)
#repo1.add_to_collaborators('ClaireJeynes1', permission='admin')

In [40]:
if bbinsightteam.get_repo_permission(repo1)==None:
    bbinsightteam.add_to_repos(repo1)
else:
    if bbinsightteam.get_repo_permission(repo1).admin==False:
        print('Not admin')
        bbinsightteam.set_repo_permission(repo=repo1,permission='admin')


In [41]:
bbinsightteam.get_repo_permission(repo1).admin

In [99]:
help(r)

In [19]:
# CURRENTLY ONLY CAPTURING `a.b.c` format of GCP names, not `b.c` without a project name
def find_gcp_table_names(find_string,text_file):
    return list(set([re.sub(find_string,"", x)[1:-1] for x in re.findall(find_string+"`[a-z0-9_-]{0,100}\.{0,1}[a-z0-9_-]*\.[a-z0-9_-]*`",text_file.lower())] \
                    +[re.sub(find_string,"", x)[1:-1] for x in re.findall(find_string+"`{0,1}[a-z0-9_-]*\.[a-z0-9_-]*`",text_file.lower())]))

def extract_edges (query_text):
    to_tables=find_gcp_table_names("create or replace table ",query_text)+\
    find_gcp_table_names("create table ",query_text)+\
    find_gcp_table_names("create or replace view ",query_text)+\
    find_gcp_table_names("create view ",query_text)+\
    find_gcp_table_names("insert ",query_text)+\
    find_gcp_table_names("insert into ",query_text)+\
    find_gcp_table_names("delete from ",query_text)+\
    find_gcp_table_names("delete ",query_text)+\
    find_gcp_table_names("update ",query_text)+\
    find_gcp_table_names("merge into ",query_text)+\
    find_gcp_table_names("merge ",query_text)

    #Might match on the delete from... can handle later with a from != to
    from_tables=find_gcp_table_names("from ",query_text)+\
    find_gcp_table_names("join ",query_text)+\
    find_gcp_table_names("using ",query_text)+\
    find_gcp_table_names(", ",query_text)+\
    find_gcp_table_names(",",query_text) #For coding syntax where from x,y is used
    
    mapping=[]
    
    for i in to_tables:
        for j in from_tables:
            mapping.append((j,i))
    
    return mapping



In [21]:

All_edges=[]

# Loop all the repos
for repo in bbdig_repos[:5]:
    files=[file for file in repo.get_contents("") if file.name.split('.')[-1]=='sql'] # Return SQL files in a repo
    print("Repo: {}".format(repo))
    for file in files:
        file_url=file.raw_data['download_url']
        file_request=requests.get(file_url)
        if file_request.ok:
            file_contents=file_request.text
#            print(file_contents)
            file_contents_clean=" ".join(file_contents.replace('\n',' ').split(" ")) # Remove newline characters and multiple spaces to standardise code
            subscripts=file_contents_clean.split(";") # Create subscripts where the data is split into different blocks of code using semicolons
            for script in subscripts:
                #print("\n")
                #print(script)
                edges=extract_edges(script) #Function to derive to and from nodes from a SQL script
                #print(edges)# Extract all the from>>to table edges 
                if len(edges)>0:
                    All_edges.append(edges)
                    
        # Placeholder to include any ipynb or .py files
All_edges


In [ ]:
# replace itv-bde-analytics-dev and itv-bde-analytics-prd with itv-bde-analytics-*
# replace no project with # itv-bde-analytics-*



In [95]:
x=requests.get('https://raw.githubusercontent.com/ITV/bbdig_mart_entitlements/master/Account%20Deleted.sql?token=AMRDGLS5LEMXSOXYYMA63AC7NR56G')

In [96]:
x.text

In [7]:
https://raw.githubusercontent.com/ITV/bbdig_mart_entitlements

In [10]:
import github


In [20]:
mytoken=open('/home/jupyter/reusable_code/github_token.txt',"r").read()
mytoken

In [26]:

help(g)

In [43]:
y.get_repos()[0].full_name

In [55]:
help(repo1.get_dir_contents("")[0])

In [56]:

repo1.get_dir_contents("")[0].path

In [131]:
x=requests.get(repo1.get_contents("")[0].raw_data['download_url'])

In [132]:
query_text=x.text

In [133]:
query_text

In [134]:


# Remove newline characters (\n)
# Collapse multiple spaces into single spaces with " ".join(<>.split(" "))
# Separate code blocks using ";"

In [135]:
import re


In [147]:
create_tables=find_gcp_table_names("create or replace table "," ".join(query_text.replace('\n',' ').split(" ")))
create_views=find_gcp_table_names("create or replace view "," ".join(query_text.replace('\n',' ').split(" ")))
from_tables=find_gcp_table_names("from "," ".join(query_text.replace('\n',' ').split(" ")))
join_tables=find_gcp_table_names("join "," ".join(query_text.replace('\n',' ').split(" ")))
insert_tables=find_gcp_table_names("insert "," ".join(query_text.replace('\n',' ').split(" ")))

In [148]:
create_views

In [81]:
from_tables

In [ ]:
CREATE 
INSERT
INSERT INTO 
DELETE
DELETE FROM
UPDATE 
MERGE
MERGE INTO 


FROM 
JOIN
USING 
COMMA